In [2]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import cv2
import numpy as np

In [4]:
print(tf.__version__)

2.17.1


In [3]:
def blur_images(image):
  return cv2.GaussianBlur(image,(5,5),0)

In [4]:
from sklearn.datasets import fetch_lfw_people
lfw_dataset = fetch_lfw_people(min_faces_per_person=70, resize=0.4)
images = lfw_dataset.images
labels = lfw_dataset.target
label_names = lfw_dataset.target_names

In [7]:
blurred_images = np.array([blur_images(img) for img in images])
augumented_images = np.concatenate((images,blurred_images))
augumented_lables = np.concatenate((labels,labels))

In [8]:
augumented_images = augumented_images / 255.0
augumented_images = augumented_images.reshape(augumented_images.shape[0],augumented_images.shape[1],augumented_images.shape[2],1)


In [9]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(augumented_images,augumented_lables,test_size=0.2,random_state=42)

In [10]:
model = models.Sequential([
    layers.Conv2D(32,(3,3),activation='relu',input_shape=(images.shape[1],images.shape[2],1)),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64,(3,3),activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(128,(3,3),activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Flatten(),
    layers.Dense(128,activation='relu'),
    layers.Dense(len(label_names),activation='softmax')
])


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [11]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10)

# Save the model
model.save('fine_tuned_facial_recognition_model.h5')

Epoch 1/10
65/65 ━━━━━━━━━━━━━━━━━━━━ 9s 63ms/step - accuracy: 0.4260 - loss: 1.7773 - val_accuracy: 0.3973 - val_loss: 1.6983
Epoch 2/10
65/65 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.4204 - loss: 1.6886 - val_accuracy: 0.3973 - val_loss: 1.7161
Epoch 3/10
65/65 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.4192 - loss: 1.6761 - val_accuracy: 0.3973 - val_loss: 1.7153
Epoch 4/10
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.3995 - loss: 1.7098 - val_accuracy: 0.3973 - val_loss: 1.7459
Epoch 5/10
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4222 - loss: 1.6757 - val_accuracy: 0.3973 - val_loss: 1.6990
Epoch 6/10
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.4157 - loss: 1.6879 - val_accuracy: 0.3973 - val_loss: 1.7023
Epoch 7/10
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4321 - loss: 1.6513 - val_accuracy: 0.3973 - val_loss: 1.7045
Epoch 8/10
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4269 - loss: 1.6746 - val_accuracy: 0.3973 - val_loss

In [12]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    zoom_range=0.15,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Augment training data
datagen.fit(X_train)


In [14]:
print(X_train.shape, X_test.shape)

(2060, 50, 37, 1) (516, 50, 37, 1)


In [17]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout, Input

# Load pre-trained MobileNetV2
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(50, 50, 3))
for layer in base_model.layers:
    layer.trainable = False  # Freeze pre-trained layers

# Add custom layers for fine-tuning
x = Flatten()(base_model.output)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(len(set(y_train)), activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=output)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


<ipython-input-17-3cad33e4a5a7>:6: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(50, 50, 3))


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


In [18]:
from tensorflow.image import resize
X_train = np.array([resize(img, (50, 50)).numpy() for img in X_train])
X_test = np.array([resize(img, (50, 50)).numpy() for img in X_test])


In [19]:
X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0


In [20]:
print(X_train.shape, X_test.shape)


(2060, 50, 50, 1) (516, 50, 50, 1)


In [21]:
import numpy as np

# Convert grayscale (50, 50, 1) to RGB (50, 50, 3)
X_train = np.repeat(X_train, 3, axis=-1)  # Repeat the single channel 3 times
X_test = np.repeat(X_test, 3, axis=-1)

print(X_train.shape, X_test.shape)  # Should now be (2060, 50, 50, 3) and (516, 50, 50, 3)


(2060, 50, 50, 3) (516, 50, 50, 3)


In [22]:
X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0


In [23]:

history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=20,
    batch_size=32
)


Epoch 1/20
65/65 ━━━━━━━━━━━━━━━━━━━━ 18s 152ms/step - accuracy: 0.2803 - loss: 2.8284 - val_accuracy: 0.3973 - val_loss: 1.7143
Epoch 2/20
65/65 ━━━━━━━━━━━━━━━━━━━━ 7s 10ms/step - accuracy: 0.3984 - loss: 1.7656 - val_accuracy: 0.3973 - val_loss: 1.7281
Epoch 3/20
65/65 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.4007 - loss: 1.7619 - val_accuracy: 0.3973 - val_loss: 1.7193
Epoch 4/20
65/65 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.4216 - loss: 1.7298 - val_accuracy: 0.3973 - val_loss: 1.7016
Epoch 5/20
65/65 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.4198 - loss: 1.7170 - val_accuracy: 0.3973 - val_loss: 1.7233
Epoch 6/20
65/65 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.4179 - loss: 1.7311 - val_accuracy: 0.3973 - val_loss: 1.7007
Epoch 7/20
65/65 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.4106 - loss: 1.7249 - val_accuracy: 0.3973 - val_loss: 1.7053
Epoch 8/20
65/65 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.4249 - loss: 1.7189 - val_accuracy: 0.3973 - val_

In [24]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
)

datagen.fit(X_train)

history = model.fit(
    datagen.flow(X_train, y_train, batch_size=32),
    validation_data=(X_test, y_test),
    epochs=20
)


Epoch 1/20


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


65/65 ━━━━━━━━━━━━━━━━━━━━ 3s 44ms/step - accuracy: 0.4172 - loss: 1.6873 - val_accuracy: 0.3973 - val_loss: 1.7024
Epoch 2/20
65/65 ━━━━━━━━━━━━━━━━━━━━ 4s 32ms/step - accuracy: 0.4057 - loss: 1.6965 - val_accuracy: 0.3973 - val_loss: 1.7024
Epoch 3/20
65/65 ━━━━━━━━━━━━━━━━━━━━ 3s 31ms/step - accuracy: 0.4291 - loss: 1.6714 - val_accuracy: 0.3973 - val_loss: 1.7038
Epoch 4/20
65/65 ━━━━━━━━━━━━━━━━━━━━ 2s 35ms/step - accuracy: 0.3990 - loss: 1.7126 - val_accuracy: 0.3973 - val_loss: 1.7057
Epoch 5/20
65/65 ━━━━━━━━━━━━━━━━━━━━ 3s 47ms/step - accuracy: 0.4055 - loss: 1.6898 - val_accuracy: 0.3973 - val_loss: 1.7038
Epoch 6/20
65/65 ━━━━━━━━━━━━━━━━━━━━ 4s 32ms/step - accuracy: 0.4114 - loss: 1.6942 - val_accuracy: 0.3973 - val_loss: 1.7013
Epoch 7/20
65/65 ━━━━━━━━━━━━━━━━━━━━ 2s 31ms/step - accuracy: 0.4147 - loss: 1.6901 - val_accuracy: 0.3973 - val_loss: 1.7012
Epoch 8/20
65/65 ━━━━━━━━━━━━━━━━━━━━ 2s 33ms/step - accuracy: 0.4098 - loss: 1.7069 - val_accuracy: 0.3973 - val_loss: 1.

In [25]:
model.save('fine_tuned_facial_recognition_model.h5')

In [28]:
# Load image from file (replace with your test image path)
def load_image_from_file(image_path):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)  # Read as grayscale
    image = cv2.resize(image, (50, 50))  # Resize image
    image = np.repeat(image[..., np.newaxis], 3, axis=-1)  # Convert to 3 channels
    image = image / 255.0  # Normalize to [0, 1]
    return image.reshape(1, 50, 50, 3)  # Reshape for model input

# Test with a sample ID image and webcam image
id_image_path = '/content/Nikil passport_1.jpg'  # Replace with your ID image path
id_image = load_image_from_file(id_image_path)

# Replace webcam image with another sample image for testing
test_image_path = '/content/DSC_0986 copy.jpg'  # Replace with another test image
webcam_image = load_image_from_file(test_image_path)

match_percentage = match_faces(id_image, webcam_image)
if match_percentage > 85:
    print(f"Match Successful! Match Percentage: {match_percentage:.2f}%")
else:
    print(f"Match Failed. Match Percentage: {match_percentage:.2f}%")


ValueError: cannot reshape array of size 7500 into shape (1,50,50,1)

In [30]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model

# Load pre-trained model
model = load_model('fine_tuned_facial_recognition_model.h5')

# Convert grayscale image (50, 50, 1) to RGB (50, 50, 3)
def load_image_from_file(image_path):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)  # Read as grayscale
    image = cv2.resize(image, (50, 50))  # Resize image
    image = np.repeat(image[..., np.newaxis], 3, axis=-1)  # Convert to 3 channels
    image = image / 255.0  # Normalize to [0, 1]
    return image.reshape(1, 50, 50, 3)  # Reshape for model input

# Load ID and test images
id_image_path = '/content/Nikil passport_1.jpg'  # Replace with your ID image path
id_image = load_image_from_file(id_image_path)

# Replace with the actual path for a test face image
test_image_path = '/content/DSC_0986 copy.jpg'  # Replace with your test image path
webcam_image = load_image_from_file(test_image_path)

# Perform face matching
def match_faces(id_image, webcam_image):
    id_image = id_image / 255.0  # Normalize to [0, 1]
    webcam_image = webcam_image / 255.0

    # Predict embeddings (features) for both ID and webcam images
    id_embedding = model.predict(id_image)
    webcam_embedding = model.predict(webcam_image)

    # Calculate cosine similarity between the embeddings
    similarity = np.dot(id_embedding, webcam_embedding.T) / (
        np.linalg.norm(id_embedding) * np.linalg.norm(webcam_embedding)
    )

    return similarity.item() * 100  # Percentage match

# Calculate match percentage
match_percentage = match_faces(id_image, webcam_image)
if match_percentage > 85:
    print(f"Match Successful! Match Percentage: {match_percentage:.2f}%")
else:
    print(f"Match Failed. Match Percentage: {match_percentage:.2f}%")


1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
Match Successful! Match Percentage: 100.00%


In [ ]:
facial-recognition-bucket-nk

In [1]:
print(tf.__version__)"

SyntaxError: unterminated string literal (detected at line 1) (<ipython-input-1-d4085be98b5b>, line 1)

In [ ]:
b